In [ ]:
import pandas as pd
import cv2
import os
import data_augment_tool
import csv
df=pd.read_csv("data//label.csv")
filename=df["filename"]
category=df["category"]
if os.path.exists("./data_augmentation") is False:
      os.makedirs("./data_augmentation")
with open("data_augmentation//label.csv","a+",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename","category"])
    for i in range(0,len(filename)):
        label=category[i]
        name=filename[i]
        img_path=os.path.join("data//",name)
        img=cv2.imread(img_path)
        
 
        cv2.imwrite(os.path.join("data_augmentation//",name),img)
        writer.writerow([name,label])
        
        after=data_augment_tool.colorjitter(img,"b")
        after_name="b_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.colorjitter(img,"s")
        after_name="s_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.colorjitter(img,"c")
        after_name="c_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.noisy(img,"gauss")
        after_name="gauss_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.noisy(img,"sp")
        after_name="sp_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.filters(img,"blur")
        after_name="blur_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.filters(img,"gaussian")
        after_name="gaussian_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])

        after=data_augment_tool.filters(img,"median")
        after_name="median_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])
        
        after=data_augment_tool.h_mirror(img)
        after_name="mirrorh_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])
        
        
        after=data_augment_tool.v_mirror(img)
        after_name="mirrorv_"+name
        cv2.imwrite(os.path.join("data_augmentation//",after_name),after)
        writer.writerow([after_name,label])
        
        




In [ ]:
import pandas as pd
import cv2
import os
import data_augment_tool
import csv
df=pd.read_csv("data_augmentation//label.csv")
if os.path.exists("./train") is False:
      os.makedirs("./train")
for i in range(0,len(df)):
    sub=df.loc[i]
    filename=sub["filename"]
    category=str(sub["category"])
    floder=os.path.join("train",category)
    if not(os.path.exists(floder)):
        os.mkdir(floder)
    img=cv2.imread(os.path.join("data_augmentation",filename))
    cv2.imwrite(os.path.join(floder,filename),img)


In [ ]:


import os
import argparse

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from my_dataset import MyDataSet
from model import swin_large_patch4_window12_384_in22k as create_model
from utils import read_split_data,train_one_epoch, evaluate

if __name__ == '__main__':
  data_path="train"
  weight_path="./swin_large_patch4_window12_384_22k.pth"
  device="cuda:0"
  num_classes=219
  epochs=100
  batch_size=2
  lr=2e-6
  
  
  device = torch.device(device if torch.cuda.is_available() else "cpu")
  if os.path.exists("./weights") is False:
      os.makedirs("./weights")
      
  tb_writer = SummaryWriter()
  train_images_path, train_images_label, val_images_path, val_images_label = read_split_data(data_path)
  img_size = 384
  data_transform = {
      "train": transforms.Compose([transforms.RandomResizedCrop(img_size),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
      }
  train_dataset = MyDataSet(images_path=train_images_path,
                                images_class=train_images_label,
                                transform=data_transform["train"])
  nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])
  train_loader = torch.utils.data.DataLoader(train_dataset,
                                                 batch_size=batch_size,
                                                 shuffle=True,
                                                 pin_memory=True,
                                                 num_workers=nw,
                                                 collate_fn=train_dataset.collate_fn)
  
  model = create_model(num_classes=num_classes).to(device)
  
  
  if weight_path != "":
      assert os.path.exists(weight_path), "weights file: '{}' not exist.".format(weight_path)
      weights_dict = torch.load(weight_path, map_location=device)["model"]
      # 删除有关分类类别的权重
      for k in list(weights_dict.keys()):
          if "head" in k:
              del weights_dict[k]
      print(model.load_state_dict(weights_dict, strict=False))
      
  pg = [p for p in model.parameters() if p.requires_grad]
  optimizer = optim.AdamW(pg, lr=lr, weight_decay=1e-7)
  for epoch in range(epochs):
      # train
      train_loss, train_acc = train_one_epoch(model=model,
                                              optimizer=optimizer,
                                              data_loader=train_loader,
                                              device=device,
                                              epoch=epoch)
  
  
  
      torch.save(model.state_dict(), "./weights/model-{}.pth".format(epoch))






OK


In [ ]:

import os
import json
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from model import swin_large_patch4_window12_384_in22k as create_model
import numpy as np
import csv
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x
def test():
    test_path = "./test"
    json_path = './class_indices.json'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    num_classes = 219
    img_size = 384
    data_transform = transforms.Compose([transforms.Resize([img_size,img_size]),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
        )

    json_file = open(json_path, "r")
    class_indict = json.load(json_file)
    class_list=list(class_indict.keys())
    class_list.append("img")
    model = create_model(num_classes=num_classes).to(device)
    list_= ["55","57","62"]
    for item in list_:
        print("model: %s\n"%item)
        model_name = 'model-%s'%item
        model_weight_path = "./weights/%s.pth"%model_name
        model.load_state_dict(torch.load(model_weight_path, map_location=device))
        model.eval()
        model.cuda()
        file = os.listdir(test_path)
        number=0
        with open("result//"+item+'.csv', 'a+', newline='') as student_file:
            writer = csv.writer(student_file)
            writer.writerow(class_list)
            for img_name in file:
                if number%1000==0:
                    print(number)
                img_path=os.path.join(test_path,img_name)
                img = Image.open(img_path)
                img = data_transform(img)
                img = torch.unsqueeze(img, dim=0)
                with torch.no_grad():
                    output = torch.squeeze(model(img.to(device))).cpu()
                ensemble_output = output.numpy()
                ensemble_output=list(ensemble_output)
                ensemble_output.append(img_name)
                writer.writerow(ensemble_output)
                number+=1
if __name__ == '__main__':
    if os.path.exists("./result") is False:
        os.makedirs("./result")
    test()
    
    json_path = './class_indices.json'
    json_file = open(json_path, "r")
    class_indict = json.load(json_file)
    class_list=list(class_indict.keys())
    df_55=pd.read_csv("result//55.csv")
    df_57=pd.read_csv("result//57.csv")
    df_62=pd.read_csv("result//62.csv")
    
    with open("result//55_57_62_Sub.csv", 'a+', newline='') as student_file:
        writer = csv.writer(student_file)
        writer.writerow(["filename","category"])
        for i in range(0,len(df_57)):
            sub_55=df_55.loc[i][class_list]
            sub_55=np.array(sub_55).astype("float32")
            sub_57=df_57.loc[i][class_list]
            sub_57=np.array(sub_57).astype("float32")
            sub_62=df_62.loc[i][class_list]
            sub_62=np.array(sub_62).astype("float32")

            
            sub=(sub_62+sub_57+sub_55)/3
            sub=torch.from_numpy(sub).cpu()
            predict = torch.softmax(sub, dim=0)
            predict_cla = torch.argmax(predict).numpy()
            predict=class_indict[str(predict_cla)]
            writer.writerow([df_57.loc[i]["img"],predict])





24089 images were found in the dataset.
24089 images for training.
0 images for validation.
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['layers.0.blocks.1.attn_mask', 'layers.1.blocks.1.attn_mask', 'layers.2.blocks.1.attn_mask', 'layers.2.blocks.3.attn_mask', 'layers.2.blocks.5.attn_mask', 'layers.2.blocks.7.attn_mask', 'layers.2.blocks.9.attn_mask', 'layers.2.blocks.11.attn_mask', 'layers.2.blocks.13.attn_mask', 'layers.2.blocks.15.attn_mask', 'layers.2.blocks.17.attn_mask'])
[train epoch 2] loss: 0.349, acc: 0.949:  19%|█▊        | 2233/12045 [20:25<1:29:46,  1.82it/s]


KeyboardInterrupt: ignored